# Create Bus to Balancing Authority area Mapping via County

In [ ]:
import json

from powersimdata.input.grid import Grid
from prereise.gather.demanddata.eia.map_ba import map_buses_to_county

Load the grid.

In [ ]:
grid = Grid(['Eastern'])

## 1. Perform mapping
The following step takes about 3 hours to run.

In [ ]:
bus_ba_map = grid.bus[grid.bus["Pd"] > 0][["Pd", "lat", "lon"]].copy()
bus_ba_map, bus_no_county_match = map_buses_to_county(bus_ba_map)

Then check if there are buses where county is null. 

In [ ]:
bus_no_county_match

The results are stored in `bus_ba_map.csv` at this stage to avoid unnecessary rerun.

In [ ]:
bus_ba_map.to_csv('bus_ba_map.csv')

Load file that lists the counties in each BA area territory.

In [ ]:
data = json.load(open('../../../../data/ba_to_county.txt'))
ba_county_list = {}
for val in data['groups'].values():
    ba_county_list[val['label']] = set(val['paths'])

Clean up some county names.

In [ ]:
for index,row in bus_ba_map.iterrows():
    for ba, clist in ba_county_list.items():
        try:
            county = row['County'].replace(' ', '_')
            county = county.replace('.', '')
            county = county.replace('-', '')
            county = county.replace('\'', '_')
            if row['County'] == 'LaSalle__IL':
                county = 'La_Salle__IL'
            if row['County'] == 'Lac Qui Parle__MN':
                county = 'Lac_qui_Parle__MN'
            if row['County'] == 'Baltimore__MD':
                county = 'Baltimore_County__MD'
            if row['County'] == 'District of Columbia__DC':
                county = 'Washington__DC'
            if row['County'] == 'St. Louis City__MO':
                county = 'St_Louis_Co__MO'
            if county in clist:
                bus_ba_map.loc[index, 'BA'] = ba
                break
        except:
            continue
bus_ba_map

Check if there are buses in the grid for which there is no BA associated. This happens to bus located outside the United States. These will be fixed manually by assigning the nearest county to them.

In [ ]:
bus_no_ba_match = list(bus_ba_map[~bus_ba_map['BA'].astype(bool)].index)
bus_no_ba_match = list(bus_ba_map[bus_ba_map['BA'].isna()].index)
bus_no_ba_match

Add zone name into the data frame for reference.

In [ ]:
bus_ba_map.loc[:, 'zone_name'] = grid.bus[grid.bus['Pd'] > 0]['zone_id'].apply(lambda x: grid.id2zone[x])

Fix mismatch county names in Virginia, West Virginia and Maryland

In [ ]:
for i in bus_no_ba_match:
    if bus_ba_map.loc[i, 'zone_name'] in {'Virginia Mountains', 'West Virginia', 'Virginia Tidewater', 'Maryland'}:
        bus_ba_map.loc[i, 'BA'] = 'PJM'

## 2. Manually assign outliers (outside US territory) to the nearest Balancing Authority
Bus with no county match:  
91: ISNE,  
7991: NYIS,  
7992: NYIS,  
8707: NYIS,  
8708: NYIS,  
40644: MISO.

In [ ]:
bus_ba_map.loc[91, "BA"] = "ISNE"
bus_ba_map.loc[91, "County"] = "Aroostook__ME"
bus_ba_map.loc[7991, "BA"] = "NYIS"
bus_ba_map.loc[7991, "County"] = "Erie__NY"
bus_ba_map.loc[7992, "BA"] = "NYIS"
bus_ba_map.loc[7992, "County"] = "Erie__NY"
bus_ba_map.loc[8707, "BA"] = "NYIS"
bus_ba_map.loc[8707, "County"] = "Niagara__NY"
bus_ba_map.loc[8708, "BA"] = "NYIS"
bus_ba_map.loc[8708, "County"] = "Niagara__NY"
bus_ba_map.loc[40644, "BA"] = "MISO"
bus_ba_map.loc[40644, "County"] = "Wayne__MI"

Other buses with no BA are set to SWPP.

In [ ]:
for i in bus_no_BA_match:
    bus_ba_map.loc[i,'BA'] = 'SWPP'

Assign buses in ERCOT Texas to SWPP or MISO based on the location by observation.

In [ ]:
miso_tx = bus_ba_map[(bus_ba_map['BA'] == 'ERCOT Texas') & 
                     (bus_ba_map['zone_name'] == 'East Texas') & 
                     ((bus_ba_map['County'] == 'Montgomery__TX') | 
                      (bus_ba_map['County'] == 'Walker__TX'))].index
for i in bus_ba_map[bus_ba_map['BA'] == 'ERCOT Texas'].index:
    if i in miso_tx:
        bus_ba_map.loc[i, 'BA'] = 'MISO'
    else:
        bus_ba_map.loc[i, 'BA'] = 'SWPP'

Make BA code cosnistent with EIA data source.

In [ ]:
ba_code_fix = {'ISONE': 'ISNE', 'NYISO': 'NYIS'}
bus_ba_map.replace(ba_code_fix, inplace=True)

Overwrite the final results with two new columns for each row (bus): BA and zone_name.

In [ ]:
bus_ba_map.to_csv('bus_ba_map.csv')